# Tutorial 6a - Case study: substitutional and interstitial defects in ZSM-5 zeolite


```
      _
    /|_|\   
   / / \ \  
  /_/   \_\  
  \ \   / /  
   \ \_/ /  
    \|_|/  

```
SOPRANO: a Python library for generation, manipulation and analysis of large batches of crystalline structures


*Developed within the CCP-NC project. Copyright STFC 2022*

---

In zeolites such as ZSM-5, some Si atoms can be replaced by Al atoms. Because of the difference in charge between Si and Al, this creates a charge imbalance in the structure, which is compensated by the presence of e.g. Na<sup>+</sup> ions in interstitial regions. In this notebook we'll look at how you can use Soprano to generate and analyse structures with such defects.


Specifically, we look at adding Al<sub>Si</sub> substitutional defects to a ZSM-5 zeolite structure, and then adding Na<sub>i</sub> counter ion defects to the resulting structures. 

We will use the `substitutionGen` and `additionGen` functions built into Soprano, and the `defects` module to analyse the resulting structures.

The first step is to import the required modules:

In [1]:
# Basic imports
import os, sys
sys.path.insert(0, os.path.abspath('..')) # This to add the Soprano path to the PYTHONPATH
                                          # so we can load it without installing it

from typing import Union

# The following lines are helpful in case we're making changes to Soprano
# and want to reload them without restarting the kernel
%reload_ext autoreload
%autoreload 2

In [2]:
# Other useful imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib widget

# Lots of useful things in ASE
import ase
from ase.io import read, write
from ase import Atoms
from ase.build import molecule
from ase.visualize import view
from ase.utils.structure_comparator import SymmetryEquivalenceCheck
from ase.geometry.geometry import find_mic
from ase.neighborlist import neighbor_list
from ase.geometry.analysis import Analysis
from ase.data.pubchem import pubchem_atoms_search
from ase.db import connect

from ase.calculators.cp2k import CP2K
from ase.optimize import BFGSLineSearch, FIRE, GPMin, MDMin, BFGS
from ase.constraints import FixAtoms


import soprano
from soprano.properties import AtomsProperty
from soprano.properties.linkage import Bonds
from soprano.collection import AtomsCollection
from soprano.collection.generate import substitutionGen, additionGen
from soprano.selection import AtomSelection
from soprano.analyse.phylogen import PhylogenCluster, Gene

import scipy
from scipy.cluster.hierarchy import dendrogram
from scipy.special import comb

import hashlib





These are the versions of the main libraries used in this notebook:

In [3]:
print('ASE version:', ase.__version__)
print('Soprano version:', soprano.__version__)
print('Pandas version:', pd.__version__)
print('Numpy version:', np.__version__)
print('Scipy version:', scipy.__version__)

ASE version: 3.22.1
Soprano version: 0.8.13
Pandas version: 1.5.1
Numpy version: 1.23.4
Scipy version: 1.9.3


In [4]:
def hash_structure(atoms):
        """A function to create an identifying hash for a given Atoms system.

        Takes into account both species and positions of atoms.

        The positions are rounded to 3 decimal places to avoid floating point
        errors.

        """

        # get the chemical symbols
        symbols = atoms.get_chemical_symbols()

        # get the positions
        positions = atoms.get_positions()

        # create a list of strings for each atom
        atom_strings = [f"{symbol} {position[0]:.3f} {position[1]:.3f} {position[2]:.3f}" 
                        for symbol, position in zip(symbols, positions)]

        # sort the strings
        atom_strings.sort()

        # create a hash from the strings
        hash = hashlib.md5("".join(atom_strings).encode('utf-8')).hexdigest()

        return hash

### Step 1: Defining the rules for the defects

We can define some functions that will check if structures that are generated are what we want. In this case, there are some rules that we want to enforce, such as Lowestein's rule, and others that will hopefully give us more realistic starting structures, such as the Na<sub>i</sub> ions being close to the Al<sub>Si</sub> ions.


The Al substitutional sites have to obey the following rules:
 - Lowenstein's rule: −Al–O–Al– bond formation is forbidden

The Na interstitial sites have to obey the following rules:
 - They must be within a certain radius of an Al substitutional site (one per Al)
 - They must not be too close (e.g. < 0.8 Å) to any other atom
 - They must be in a pore. This is defined roughly as having at more 4 O atoms within 2.8 Å of the Na atom (this can be easily refined, but seems to work reasonably well).

Then there are some ideas for how we might rank the most realistic structures based:

- The Al<sub>Si</sub> substitutional sites should spaced out as much as possible (no clustering).
- The Na<sub>i</sub> interstitial sites should as far as possible from each other (while still being close to the Al<sub>Si</sub> sites).

We start by defining a few useful functions that capture the rules above.




We can use the AtomsProperty class to define a new property based on the minimum distance between any two atoms of the same species. We can then use this property to rule out or rank the structures.

In [67]:

class Min_X_X_Distance(AtomsProperty):
    """
    Minimum distance between two atoms of the same element X
    """
    default_name = 'minXXdist'
    default_params = {}

    @staticmethod
    def extract(s, X:str): # This is where the core of the calculation happens
        # s is a single Atoms object passed to this method
        # X is a string passed to this method: it is the element we are interested in
        
        # Get the indices of the X atoms
        xsel = AtomSelection.from_element(s, X)
        Xindices = xsel.indices
        min_dist = np.zeros(len(Xindices))
        for i, idx in enumerate(Xindices):
            other_X = np.delete(Xindices, i)
            min_dist[i] = s.get_distances(idx, other_X, mic=True).min()
        return min_dist.min()
    

class XYCoordination(AtomsProperty):
    """
    Compute number of Y neighbors for each X atom
    """
    default_name = 'XYcoord'
    default_params = {'rcut': 2.8}

    @staticmethod
    def extract(s:Atoms, X:str, Y:str, rcut:float): # This is where the core of the calculation happens
        # s is a single Atoms object passed to this method
        # X is a string passed to this method: it is the element we are interested in
        # Y is a string passed to this method: it is the element of the neighbors we are interested in
        
        # Get the indices of the X atoms
        xsel = AtomSelection.from_element(s, X)
        ysel = AtomSelection.from_element(s, Y)

        sel = xsel + ysel
        subset = sel.subset(s)

        # neighbour list
        i = neighbor_list('i', subset, rcut)
        coord = np.bincount(i)
        idx = AtomSelection.from_element(subset, X).indices
        coord = coord[idx]
        return coord

We can also define a function that will check if a given atom is in a pore or not. 

One way I considered was to define pore regions, then check if the added atom is one such region. For example:

```python
PORE_DIAMETER = 9 # Angstroms
pore_radius = PORE_DIAMETER / 2
PORE_CENTRES_AC = [[0.5, 0], [0,0.5]]
def is_in_pore(atoms:Atoms, idx, pore_centres=PORE_CENTRES_AC, pore_radius=pore_radius):
    '''
    Check if an atom is in the pore
    atoms: ASE atoms object
    idx: index of the atom to check
    pore_centres: list of pore centres
    pore_radius: radius of the pore
    '''
    #atoms a and c scaled positions
     
    atom_position = atoms.positions[idx]
    pore_centre_xz = [atoms.cell.cartesian_positions([c[0], 0, c[1]]) for c in pore_centres]
    vector_to_centres = np.array(pore_centre_xz) - np.array(atom_position)
    # ignore y component
    vector_to_centres[:,1] = 0
    # get minimum image convention distances
    d = [find_mic(v, atoms.cell)[1] for v in vector_to_centres] 
    # check if any of the distances are less than the pore radius
    return np.any(np.array(d) < pore_radius)


# and then a function for the rule:

def in_pore_rule(atoms: Atoms, inds_to_check):
    '''
    Checks to make sure that inds_to_check are not in the pore.
    '''
    for idx in inds_to_check:
        if not is_in_pore(atoms, idx):
            return False
    return True
```

But this is a bit of a pain to define, and it's not very general. 


**Instead**, we can just check if there are a reasonable number of O atoms within a certain distance of the added atom.

In [68]:
def in_pore_rule(atoms:Atoms, X:str='Na', Y:str='O', rcut=3.0, threshold:int=4)->bool:
    """
    checks if all X atoms have at most threshold Y neighbors
    """
    xycoord = XYCoordination().get(atoms, X=X, Y=Y, rcut=rcut)
    return np.all(xycoord <= threshold)


Now we can use these functions to define rules for accepting or rejecting structures.

In [69]:
# ------------------ Al Rule ------------------ #
def lowenstein_rule(atoms: Atoms, indices: list):
    '''
    The Lowenstein rule states Al-O-Al bonds are forbidden in zeolites.

    Here's a very naive implementation based on a hard-coded distance threshold.

    A more sophisticated implementation would use the bond matrix.
    '''
    RCUT = 4.0 # Angstroms

    if len(indices) < 2:
        # no need to check, return True
        return True
    
    # get the minimum distance between two Al atoms
    min_dist = Min_X_X_Distance.extract(atoms, X='Al')
    # if the maximum distance is smaller than RCUT, return False
    if min_dist < RCUT:
        return False
    return True

# ------------------ Na Rules ------------------ #
def overlap_rule(
        atoms: Atoms,
        inds_to_check, 
        rcut = 0.7, # Angstroms
        )-> bool:
    '''
    Checks to make sure that inds_to_check do not overlap with any other atoms
    in the atoms object.
    '''
    for idx in inds_to_check:
        # all indices except the one we are checking
        other_indices = np.delete(np.arange(len(atoms)), idx)
        dists = atoms.get_distances(idx, other_indices, mic=True)
        if np.any(dists < rcut):
            return False
    return True




def Na_min_sep_rule(atoms: Atoms, inds_to_check, rcut = 3.5):
    '''
    Checks to make sure that inds_to_check are at least rcut apart.
    '''
    for i, idx in enumerate(inds_to_check):
        # all indices except the one we are checking
        other_indices = np.delete(inds_to_check, i)
        dists = atoms.get_distances(idx, other_indices, mic=True)
        if np.any(dists < rcut):
            return False
    return True



# Combine the no_overlap and in_pore rules into a single rule
def Na_accept_rule(atoms: Atoms, indices: list):
    '''
    A series of rules that determine whether to accept a configuration generated
    by the additionGen.

    Rule 1: No overlap with other atoms
    Rule 2: No Na in the pore
    '''
    Na_idx = AtomSelection.from_element(atoms, 'Na').indices
    # Rule 1
    if not overlap_rule(atoms, Na_idx):
        return False
    # Rule 2
    if not in_pore_rule(atoms, X='Na', Y='O', threshold=4):
        return False
    # Rule 3
    if not Na_min_sep_rule(atoms, Na_idx):
        return False
    
    return True


In [70]:

def get_unique_structures(coll_orig: AtomsCollection, stol:float=0.05, indices:Union[list,None]=None):
    '''
    Given an AtomsCollection, return a new AtomsCollection with only the unique structures.
    This is a naive implementation that compares all structures with a list of unmatched structures.

    coll_orig: AtomsCollection
    stol: symmetry tolerance
    indices: indices of the atoms to compare. If None, all atoms are compared.
    
    '''
    if len(coll_orig) < 2:
        return coll_orig
    
    if indices is None:
        indices = np.arange(len(coll_orig.structures[0]))
    comp = SymmetryEquivalenceCheck(to_primitive=False, stol=stol)
    unmatched = np.ones(len(coll_orig), dtype=bool)
    unique = np.zeros(len(coll_orig), dtype=bool)
    # make a copy of the collection
    coll = AtomsCollection(coll_orig.structures)
    for i, s in enumerate(coll.structures):
        if unmatched[i]:
            unique[i] = True
            for j in range(i+1, len(coll)):
                if unmatched[j]:
                    if comp.compare(s[indices], coll.structures[j][indices]):
                        unmatched[j] = False
    return coll_orig[unique]

### Step 2: substituting some Si for Al

Now we can read in the base structure and generate the Al substitutional defects. 

We must choose the number of Si atoms to substitute for Al. In ZSM-5, the number of Si atoms is 96, but in order to satisfy Lowenstein's rule, the maximum number of Al atoms is much lower than this. 

We can use the `substitutionGen` function to generate all possible structures with a given number of Al atoms. We can then use the rules we defined above to filter out the structures we don't want.


In [71]:

# path to the cif file
cif_path = 'tutorial_data/ZSM-5.cif'

# number of Al atoms to substitute
NAl = 2


# read in the cif file
zsm5_original = read(cif_path)
# make a copy of the original structure - just in case
zsm5 = zsm5_original.copy()

# We want to substitute the silicon atoms in the structure with aluminum
# First, we select the silicon atoms
si_sel = AtomSelection.from_element(zsm5, 'Si')

print('Number of Si atoms: {0}'.format(len(si_sel)))
print('Number of Al atoms to substitute in: {0}'.format(NAl))
# The total number of possible structures is given by the combination formula
print(f'There are {comb(len(si_sel), NAl):.3e} possible structures')

# Then, we generate the structures, setting the acceptance rule to the Lowenstein rule
# and the maximum number of attempts to max_attempts so that the combinatorial explosion doesn't wreck our computer
# random is set to True so that the structures are generated in random order (necessary given the large number
# of possible structures).
max_attempts = 5000
aG = substitutionGen(
    zsm5,
    'Al',
    to_replace=si_sel,
    n=NAl,
    accept=lowenstein_rule,
    max_attempts=max_attempts,
    random=False
    )
# Collect the structures in an AtomsCollection
aColl = AtomsCollection(aG)
# With max_attempts random attempts, we get this many structures that satisfy the acceptance rule
print(f'{len(aColl)} structures generated from a max of {max_attempts} attempts.')
print(f'(i.e. {max_attempts - len(aColl)} were rejected by the acceptance rule)')


Number of Si atoms: 96
Number of Al atoms to substitute in: 2
There are 4.560e+03 possible structures
4368 structures generated from a max of 5000 attempts.
(i.e. 632 were rejected by the acceptance rule)


We can optionally order the structures based on e.g. the minimum Al-Al separation. The idea behind this is that the most favourable structure are probably those for which the Al atoms are as far apart as possible (not clustered).

In [72]:
if NAl > 1:
    min_dists = np.array([Min_X_X_Distance.extract(atoms, X='Al') for atoms in aColl.structures])
    order = np.argsort(min_dists)
    # order the structures by increasing minimum Al-Al distance
    aColl = aColl[order]

If there are a reasonable* number of total structures, we can check for the unique ones by doing the following:

*reasonable depends on your patience/machine. For 96 Si with 2 Al replacements, there are 4560 structures (4368 that obey Lowestein's rule). Comparing Si and Al positions for all of these structures against every other one would take over 2 hours on my machine (I killed it after 2 hours...).


In [73]:
# # reduce to just the unique structures
# nbefore_unique = len(aColl)
# # don't bother comparing the O atoms
# inds_to_compare = [atom.index for atom in aColl.structures[0] if atom.symbol != 'O']
# aColl_unique = get_unique_structures(aColl, stol=0.1, indices=inds_to_compare)
# print(f'The number of unique structures is {len(aColl_unique)} (compared to {nbefore_unique} without checking for uniqueness)')


We probably want to limit the total number of structures for the next steps, so let's set an upper bound for the number of structures to include here:

In [74]:
max_num_structures = 50
num_structures = min(max_num_structures, len(aColl))
aColl = aColl[-num_structures:]


At this stage, we might want to save or view the structures to check that they look reasonable. We can write them to e.g. cif file(s) using the ASE `write` function or view them using the ASE GUI.

In [75]:
# [optional] write out the Al-substituted structures to cif files
# for i, s in enumerate(aColl.structures):
#     write(f'ZSM-Al_only_{i:05d}.cif', s)

# or write out the structures to a single cif or (extended) xyz file
# write('ZSM-Al_only_all.cif', aColl.structures)
# write('ZSM-Al_only_all.xyz', aColl.structures)


# [optional] View the structures in ASE-GUI
# view(aColl.structures)

# [optional] Write the structures to an ASE database
# db = connect(f'tutorial_data/ZSM-{NAl}_Al_only.db')
# for s in aColl.structures:
#     db.write(s, key_value_pairs={'structure_hash': hash_structure(s)})


### Step 3: adding Na interstitials

Now that we have the Al-substituted structures, we can add in the Na counterions. We will use the `additionGen` function to do this. 

To better organise things, we can define a function that takes in a structure and adds the Na interstitials to it with some parameters:

In [76]:
def get_added_structure(
        atoms: Atoms,
        defect: str = 'Na',
        host: str = 'Al',
        add_r:float=2.5,
        ndefects:int=1,
        accept_rule=Na_accept_rule,
        only_best:bool=True,
        remove_duplicates:bool=True,
        stol:float=0.5,):
    '''
    Generate a structures with `defect` atoms added to the structure.
    By default we add Na atoms to the structure and accept the structures that satisfy the Na_sep_rule.

    Parameters
    ----------
    atoms : Atoms
        The structure to which we want to add the defect atoms.
    defect : str, optional
        The element of the defect atoms, by default 'Na'
    host : str, optional
        The element of the host atoms, by default 'Al'
    add_r : float, optional
        The distance from the host atoms to add the defect atoms, by default 2.5 (Angstroms)
    ndefects : int, optional
        The number of defect atoms to add per host atom, by default 1
    accept_rule : function, optional
        The rule to accept the structures, by default Na_accept_rule
    only_best : bool, optional
        Whether to return only the 'best' structure, by default True
        or return all the structures that satisfy the acceptance rule.
    remove_duplicates : bool, optional
        Whether to remove duplicate structures, by default True
        Duplicates are defined naiively as having Na atoms in the same positions (within a tolerance).
    stol : float, optional
        The tolerance for determining if two atoms are in the same position, by default 0.5 (Angstroms)
    '''
    # additionGen uses a repulsion algorithm to add the atoms
    # to avoid placing atoms on existing bonds
    # These are the parameters for the repulsion algorithm:
    rep_alg_kwargs = {
        'iters':50, # number of 'optimisation' iterations
        'attempts':30, # number of points on a sphere to try to place the atom
        'step':1e-1, # step size for the optimisation
        'simtol':stol, # tolerance for equivalent positions
        'random': False # whether to use random sphere points or an equidistant grid
    }
    # To speed things up, we can pre-compute the bond matrix for the host structure
    # This also allows us to specify custom radii for the atoms.
    # The scaling factor has been chosen so that the number of Al-O bonds is correct (4)
    radii_scaling = 2.5
    radii = {
        'Si': 0.54  * radii_scaling, # Si 4+
        'Al': 0.675 * radii_scaling, # Al 3+
        'O':  1.26  * radii_scaling, # O 2-
        }
    bonds = Bonds.get(atoms, vdw_custom=radii, store_array=True)
    # Get the structure generator
    aG = additionGen(
        atoms,
        defect,
        to_addition=AtomSelection.from_element(atoms, host),
        add_r=add_r,
        n=ndefects,
        accept=accept_rule,
        bonds=bonds,
        rep_alg_kwargs=rep_alg_kwargs,
        random=True,
        max_attempts=1000)
    
    # Collect the structures in an AtomsCollection
    aColl_added = AtomsCollection([a for a in aG])
    ngenerated = len(aColl_added)

    if ngenerated == 0:
        print('No structures accepted for this host structure.')
        return AtomsCollection([])
    
    if remove_duplicates:
        # remove duplicate structures
        Na_indices = AtomSelection.from_element(aColl_added.structures[0], defect).indices
        aColl_added = get_unique_structures(aColl_added, indices=Na_indices, stol = stol)

    if only_best:
        # get the minimum Na-Na distance for each structure
        min_dists = Min_X_X_Distance.get(aColl_added, X=defect)
        # get the index of the structure with the largest minimum Na-Na distance
        best_s_idx = np.argmax(min_dists)
        # return a collection with just the best structure
        aColl_added = aColl_added[best_s_idx]

    print(f'Generated {ngenerated: >5}; {len(aColl_added): >5} structures added to collection.')
    return aColl_added

Now we can loop over our Al-substituted structures and add the Na interstitials to them. We can use the rules we defined above to filter out the structures we don't want.

We can use the Al-substituted structures saved in the `aColl` AtomsCollection, or we can read in the structures from the database we saved earlier (`f'tutorial_data/ZSM-{NAl}_Al_only.db'`) (or CIF files if we saved them).

In [58]:
structures_to_decorate = aColl.structures

## or read in the structures from a cif 
# structures_to_decorate = read('ZSM-Al_only_all.cif', index=':')

## or (extended) xyz file
# structures_to_decorate = read('ZSM-Al_only_all.xyz', index=':')

## or read in the structures from an ASE database
# db = connect(f'tutorial_data/ZSM-{NAl}_Al_only.db')
# structures_to_decorate = [row.toatoms() for row in db.select()]

In [78]:
all_generated_structures = AtomsCollection()
for i, s in enumerate(structures_to_decorate):
    coll_temp = get_added_structure(
        s,
        defect = 'Na',
        host= 'Al',
        add_r=2.25,
        ndefects=NAl,
        accept_rule=Na_accept_rule,
        only_best=False,
        stol = 0.75)
    all_generated_structures += coll_temp
    
print(f'Accepted {len(all_generated_structures)} structures in total.')

Atom 4 has 4 bonds and 4 possible attachment points
Atom 24 has 4 bonds and 4 possible attachment points
Generated     1;     1 structures added to collection.


Atom 5 has 4 bonds and 4 possible attachment points
Atom 25 has 4 bonds and 4 possible attachment points
Generated     1;     1 structures added to collection.
Atom 0 has 4 bonds and 4 possible attachment points
Atom 6 has 4 bonds and 4 possible attachment points
Generated     1;     1 structures added to collection.
Atom 1 has 4 bonds and 4 possible attachment points
Atom 7 has 4 bonds and 4 possible attachment points
Generated     1;     1 structures added to collection.
Atom 3 has 4 bonds and 5 possible attachment points
Atom 5 has 4 bonds and 4 possible attachment points
Generated     2;     1 structures added to collection.
Atom 2 has 4 bonds and 5 possible attachment points
Atom 4 has 4 bonds and 4 possible attachment points
Generated     2;     1 structures added to collection.
Atom 50 has 4 bonds and 4 possible attachment points
Atom 52 has 4 bonds and 4 possible attachment points
Generated     1;     1 structures added to collection.
Atom 51 has 4 bonds and 4 possible attachme

At this point we're pretty much done with the generation of structures. We can save the structures to file and use them for further analysis.

In [17]:
# Write the structures to an ASE database
db = connect(f'./tutorial_data/ZSM-{NAl}Al-{NAl}Na.db')
for s in all_generated_structures.structures:
    db.write(s, key_value_pairs={'structure_hash': hash_structure(s)})


# [optional]
# write out the structures to cif files
# for i, s in enumerate(all_generated_structures.structures):
#     write(f'ZSM-Al-Na_{i:05d}.cif', s)

# or write out the structures to a single cif file
# write('ZSM-Al-Na_all.cif', all_generated_structures.structures)
# or write out the structures to a single xyz file
# write('ZSM-Al-Na_all.xyz', all_generated_structures.structures)

# [optional] View the structures in ASE-GUI
# view(all_generated_structures.structures)



In [89]:
# [optional] use the ASE web interface to view the database
# pathtodb = f'./tutorial_data/ZSM-{NAl}Al-{NAl}Na.db'
# !ase db $pathtodb -w

## Step 4: Quick calculation of defect energies


Having generated a set of candidate structures, we can now calculate the defect energies for each of them.

Becuase the structures as-generated are likely to be far from equilibrium, we will first relax them for a few steps before calculating the defect energies.

We will use the xTB functionality within CP2K, orchestrated by ASE for this, but you could use any other calculator you want. 

> Note that these are **not** converged or tested parameters, they are just a quick example to show how you might do this. 

Similarly, here we will use the ASE database to store the results, but you could use any method you want to store the results.

In [90]:
inp = """
&GLOBAL
  PREFERRED_DIAG_LIBRARY SL
&END GLOBAL
&FORCE_EVAL
   METHOD Quickstep
   &DFT
      &MGRID
            CUTOFF  100
            REL_CUTOFF  10
            NGRIDS  5
        &END MGRID
      &QS
         METHOD XTB
            &XTB
            CHECK_ATOMIC_CHARGES F
            COULOMB_INTERACTION  T
            DO_EWALD  T
            STO_NG  6 # default is 6
            DO_NONBONDED F
            &PARAMETER
              DISPERSION_PARAMETER_FILE /opt/cp2k/data/dftd3.dat
            &END PARAMETER

         &END XTB
      &END QS
      &POISSON
        &EWALD
          ALPHA 1.0
          EWALD_TYPE SPME
          GMAX 25
        &END EWALD
      &END POISSON
      &SCF
        SCF_GUESS MOPAC
        MAX_SCF  8
        EPS_SCF  1.0E-6
        &OT
            MINIMIZER  DIIS
            PRECONDITIONER FULL_SINGLE_INVERSE
        &END OT
        &OUTER_SCF
            EPS_SCF  1.0E-6
            MAX_SCF  20
        &END OUTER_SCF

      &END SCF
   &END DFT
&END FORCE_EVAL
"""
# See https://wiki.fysik.dtu.dk/ase/ase/calculators/cp2k.html for more information on the CP2K calculator
xtb_calc = CP2K(basis_set=None,
                basis_set_file=None,
                max_scf=None,
                cutoff=None,
                force_eval_method=None,
                potential_file=None,
                poisson_solver=None,
                pseudo_potential=None,
                stress_tensor=False,
                xc=None,
                command = "docker run --env OMP_NUM_THREADS=8 -u $(id -u ${USER}):$(id -g ${USER}) -v ${PWD}:/mnt --shm-size=1g -i cp2k/cp2k cp2k_shell",
                directory="tutorial_data/tmp/",
                print_level = "MEDIUM",
                label='cp2k', inp=inp)

def get_xtb_energy(atoms, label='cp2k', calc = xtb_calc):
    '''Get the single point energy of an atoms object using the provided ASE calculator'''
    calc.label = label
    atoms.calc = calc
    return atoms.get_potential_energy()

def get_relaxed_xtb_energy(
        atoms,
        label='cp2k',
        calc=xtb_calc,
        constrain=True,
        max_ionic_steps=100,
        fmax = 0.05,
        optimizer=FIRE,
        directory=None,
        ):
    '''
    Args:
        atoms (ase.Atoms): the structure to relax
        label (str): label for the calculation
        calc (ase.Calculator): the calculator to use
        constrain (bool): whether to constrain the atoms. This will fix all the atoms except the Na atoms.
                          Switch this off if you want to relax the whole structure or apply a different constraint.
        max_ionic_steps (int): maximum number of ionic steps  
        fmax (float): maximum force allowed on each atom (eV/A)
        optimizer (ase.optimize.Optimizer): the optimizer to use (FIRE, BFGS etc.)
        directory (str): the directory to run the calculation in
    Returns:
        ase.Atoms: the relaxed structure

    '''
    # constraints
    if constrain:
        # constrain all the atoms except the Na 
        Na_indices = [atom.index for atom in atoms if atom.symbol == 'Na']
        fixed_mask = [False if i in Na_indices else True for i in range(len(atoms))]
        atoms.set_constraint(FixAtoms(mask=fixed_mask))
    # attach calculator
    calc.label = label
    if directory is not None:
      calc.directory = directory
    atoms.calc = calc
    # relax
    dyn = optimizer(atoms)
    dyn.run(fmax=fmax, steps=max_ionic_steps)
    # return relaxed structure
    return atoms

    


### Run the calculations

Now we can loop over the generated structures and calculate the defect energies for each of them.

We store the results (relaxed structures and total energies) in an ASE database.

[For reference, using the CP2K XTB settings above, running on 8 threads, the calculations took about 36s per structure on an 11th Gen Intel i9-11900H.]

In [95]:
db = connect(f'./tutorial_data/ZSM-{NAl}Al-{NAl}Na.db')
for row in db.select():
    id = row.id
    atoms = row.toatoms()
    # check if the energy is already in the database
    if row.energy:
        print(f'Energy for {id} already in database, skipping...')
        continue
    print(f'Calculating energy for structure {id}')
    # update the energy
    energy = get_xtb_energy(atoms)
    db.update(id, atoms)


Energy for 1 already in database, skipping...
Energy for 2 already in database, skipping...
Energy for 3 already in database, skipping...
Energy for 4 already in database, skipping...
Energy for 5 already in database, skipping...
Energy for 6 already in database, skipping...
Energy for 7 already in database, skipping...
Energy for 8 already in database, skipping...
Energy for 9 already in database, skipping...
Energy for 10 already in database, skipping...
Energy for 11 already in database, skipping...
Energy for 12 already in database, skipping...
Energy for 13 already in database, skipping...
Energy for 14 already in database, skipping...
Energy for 15 already in database, skipping...
Energy for 16 already in database, skipping...
Energy for 17 already in database, skipping...
Energy for 18 already in database, skipping...
Energy for 19 already in database, skipping...
Energy for 20 already in database, skipping...


We can view the results in the ASE database web app:

In [94]:
# db_path = f'./tutorial_data/ZSM-{NAl}Al-{NAl}Na.db'
# !ase db -w $db_path

### Quick relaxations

As we can see, the structures have very large forces on them, so we probably wants to relax them a bit before screening for the lowest energy structures -- i.e. to get them closer to their nearest local minima.

We can do this by running a quick relaxation on each structure, using the same settings as above, for just a few ionic relaxation steps, keeping all but the Na positions fixed. We'll use a new db for that, just to keep things tidy.

In [25]:
db = connect(f'./tutorial_data/ZSM-{NAl}Al-{NAl}Na.db')
db_rlx = connect(f'./tutorial_data/ZSM-{NAl}Al-{NAl}Na_relaxed.db')

for row in db.select():
    id = row.id
    name = f'ZSM-{NAl}Al-{NAl}Na-{id:05d}'
    atoms = get_relaxed_xtb_energy(
            row.toatoms(),
            label=name,
            max_ionic_steps=20,
            optimizer=BFGS,
            constrain=True,
            directory='./tutorial_data/tmp/')

    energy = atoms.get_potential_energy()
    structure_hash = hash_structure(atoms)
    db_rlx.write(atoms, relaxed=True, key_value_pairs={'structure_hash': structure_hash})



      Step     Time          Energy         fmax
BFGS:    0 11:19:38   -30100.110809       10.6133
BFGS:    1 11:19:50   -30102.320086        5.1347
BFGS:    2 11:20:00   -30103.196844        2.2709
BFGS:    3 11:20:10   -30103.477383        1.0429
BFGS:    4 11:20:19   -30103.541937        0.9260
BFGS:    5 11:20:28   -30103.567194        0.9644
BFGS:    6 11:20:37   -30103.597611        0.8619
BFGS:    7 11:20:46   -30103.628588        0.6088
BFGS:    8 11:20:56   -30103.653848        0.5739
BFGS:    9 11:21:05   -30103.679154        0.6743
BFGS:   10 11:21:14   -30103.695964        0.7688
BFGS:   11 11:21:24   -30103.725363        0.8161
BFGS:   12 11:21:33   -30103.767533        0.6320
BFGS:   13 11:21:43   -30103.821521        0.3489
BFGS:   14 11:21:55   -30103.912608        0.3991
BFGS:   15 11:22:13   -30103.969128        0.3374
BFGS:   16 11:22:28   -30104.014979        0.2937
BFGS:   17 11:22:41   -30103.989540        0.5368
BFGS:   18 11:23:00   -30104.025581        0.2429
B

Let's look at the energy ranking of the structures before and after the quick relaxations:

In [96]:
db = connect(f'./tutorial_data/ZSM-{NAl}Al-{NAl}Na.db')
db_rlx = connect(f'./tutorial_data/ZSM-{NAl}Al-{NAl}Na_relaxed.db')

all_energies = [row.energy for row in db.select()]
all_ids = [row.id for row in db.select()]

all_energies_rlx = [row.energy for row in db_rlx.select()]
all_structures = [row.toatoms() for row in db_rlx.select()]

# sort by energy
energy_order = np.argsort(all_energies_rlx)

for idx in energy_order:
    e = all_energies[idx] - min(all_energies)
    e_rlx = all_energies_rlx[idx] - min(all_energies_rlx)
    print(f'{all_ids[idx]:05d}: {e:16.8f} {e_rlx:16.8f} eV')

00004:      18.57133693       0.00000000 eV
00001:       0.00000000       0.12043454 eV
00002:       0.01803631       0.14355143 eV
00003:      19.61942560       0.28326310 eV
00009:       0.77924298       1.43483028 eV
00005:       0.55998853       1.45882498 eV
00006:       3.20576625       1.64241234 eV
00011:       3.60295858       2.70992379 eV
00010:       3.57604515       2.71951661 eV
00008:       2.87187655       2.76296294 eV
00012:       2.88633810       2.79349499 eV
00007:       3.38149090       3.19304134 eV
00020:       5.28468681       3.51770487 eV
00019:       8.64787879       4.27988386 eV
00016:       6.47191654       4.91255105 eV
00017:       6.67457469       5.00738583 eV
00018:       6.68780493       5.05075827 eV
00013:       5.53725804       5.55769112 eV
00014:       5.57592473       5.57873935 eV
00015:       7.59118441       5.92242884 eV


Clearly we see that, while there is a correlation between the energies before and after the quick relaxations, it is not perfect. This is to be expected since the initial structures might be close to low-energy configuration, but e.g. starting out too close to another atom. 

In [ ]:
# to get a better sense of the patterns, we can look at the relaxed structures in turn, ordered by energy.
# view([all_structures[i] for i in energy_order])

## Step 5: swapping Na for a probe molecule

In some cases, we might want to replace some of all of the Na ions with a different species or molecule. 

We can do this by adding in a pre-defined molecule to the structure, aligning the molecule's centre of mass with the Na ion position. For the rotation of the molecule, here we choose to align one of the molecule's principal axes with the vector from the centre of mass to the Al atom. 



In [97]:
def get_molecule_intertial_tensor(atoms):
    """
    Calculate the inertia tensor of a molecule.
    Args:
        atoms (ase.Atoms): the molecule
    Returns:
        np.ndarray: the inertia tensor with the convention that the
                    longest axis is along the first eigenvector
    """
    # get the positions and masses
    # (assumes all are in the same unit cell)
    mol_pos = atoms.positions
    mol_ms = atoms.get_masses()

    # We still need to correct the positions with the COM
    mol_com = atoms.get_center_of_mass()
    mol_pos -= mol_com

    tens_i = (
        np.identity(3)[None, :, :]
        * np.linalg.norm(mol_pos, axis=1)[:, None, None] ** 2
    )

    tens_i -= mol_pos[:, None, :] * mol_pos[:, :, None]
    tens_i *= mol_ms[:, None, None]
    tens_i = np.sum(tens_i, axis=0)

    evals, evecs = np.linalg.eigh(tens_i)
    # General ordering convention: we want the component of the
    # longest position to be positive along evecs_0, and the component
    # of the second longest (and non-parallel) position to be positive
    # along evecs_1, and the triple to be right-handed of course.
    mol_pos = sorted(mol_pos, key=lambda x: -np.linalg.norm(x))
    if len(mol_pos) > 1:
        evecs[0] *= np.sign(np.dot(evecs[0], mol_pos[0]))
    e1dirs = np.where(np.linalg.norm(np.cross(mol_pos, mol_pos[0])) > 0)[0]
    if len(e1dirs) > 0:
        evecs[1] *= np.sign(np.dot(evecs[1], mol_pos[e1dirs[0]]))
    evecs[2] *= np.sign(np.dot(evecs[2], np.cross(evecs[0], evecs[1])))
    # Evecs must be proper
    evecs /= np.linalg.det(evecs)

    return evecs

In [98]:
def get_molecule_decorated_structure(atoms_in, molecule_in, element_to_replace='Na'):
    '''
    Args:
        atoms_in (ase.Atoms): the structure to decorate
        molecule_in (ase.Atoms): the molecule to decorate with
        element_to_replace (str): the element to replace in the structure (all such will be replaced by a molecule_in)
    Returns:
        ase.Atoms: the decorated structure
    '''
    atoms = atoms_in.copy()
    Na_positions = [atom.position for atom in atoms if atom.symbol == element_to_replace]
    mols = [molecule_in.copy() for _ in range(len(Na_positions))]

    Al_idx = [atom.index for atom in atoms if atom.symbol == 'Al']
    Na_idx = [atom.index for atom in atoms if atom.symbol == element_to_replace]


    # now we have a list of molecules and a list of Na positions
    # we can figure out where the molecules should go
    for mol, Na_i, Na_pos in zip(mols, Na_idx, Na_positions):
        mol_com = mol.get_center_of_mass()

        # make sure the molecule is centered at the origin
        mol.translate(- mol_com)

        # rotation of molecule
        # We might want to align one of the molecule's principal axes with e.g. the molecule-Al "bond"

        # nearest Al atom
        nn_al_idx = atoms.get_distances(Na_i, Al_idx, mic=True).argmin()
        v1 = atoms.get_distance(Al_idx[nn_al_idx], Na_i, mic=True, vector=True)
        v2 = get_molecule_intertial_tensor(mol)[2] # align the 3rd eigenvector with the molecule-Al bond
        mol.rotate(-v2, v1)

        # OR we could just rotate the molecule randomly like this:
        # mol.rotate(np.random.rand() * 360, v= np.random.rand(3))

        # move the molecule to the Na position
        mol.translate(Na_pos)

        # [optional] move the molecule slightly away from the Al atom
        # (this is to avoid the molecule being too close to the Al atom)
        mol.translate(v1 * 1.5)

    # # now remove all Na atoms
    # non_Na_indices = [atom.index for atom in atoms if atom.symbol != element_to_replace]
    # atoms = atoms[non_Na_indices]

    # # OR replace them with H atoms (e.g. to get a TMPOH+ if the molecule is TMPO)
    for Na_i in Na_idx:
        atoms[Na_i].symbol = 'H'

    # and add in the molecules
    for i, mol in enumerate(mols):
        # tag all the atoms in the molecule with the molecule index
        for atom in mol:
            atom.tag = i+1
        atoms.extend(mol)
    return atoms


Generating all the molecule-replaced structures is straightforward:

In [104]:
# For this example, I'll use NH4+ as the molecule. A hacky way to generate it is to use methane and swap the C for an N
# molecule_in = molecule('CH4')
# molecule_in[0].symbol = 'N'
molecule_in = pubchem_atoms_search(name="Trimethylphosphine-oxide")
# Note about the pubchem search: I occasionally got a temporary failure in name resolution.
# Trying again later worked (or manually define the molecule).
all_generated_strucutures_mol = [get_molecule_decorated_structure(atoms, molecule_in) 
                                 for atoms in all_generated_structures.structures]

In [105]:
#[optional] View the structures
view(all_generated_strucutures_mol)
# Or write them to a cif file
# write('tutorial_data/ZSM-8Al-8Na_mol.cif', all_generated_strucutures_mol)


<Popen: returncode: None args: ['/home/jks/miniconda3/bin/python', '-m', 'as...>

As before, we can do a simple energy calculation/quick geometry optimisation to rank structures. In the follow example, we just try to relax one of the structures for a few steps. 

In [119]:

mol_struc = all_generated_strucutures_mol[15].copy()
# constrain all the atoms except the molecule atoms (those tagged with a positive integer)
mol_indices = [atom.index for atom in mol_struc if atom.tag > 0]
fixed_mask = [False if i in mol_indices else True for i in range(len(mol_struc))]
mol_struc.set_constraint(FixAtoms(mask=fixed_mask))
mol_struc = get_relaxed_xtb_energy(mol_struc,
                                   label='mol_struc15.inp',
                                   constrain=False, # (otherwise the default constraints will overwrite our custom one)
                                   optimizer=BFGS,
                                   directory='tutorial_data/tmp/',
                                   max_ionic_steps=10)

      Step     Time          Energy         fmax
BFGS:    0 15:43:51   -31064.991899       48.6922
BFGS:    1 15:44:25   -31073.933448       15.6223
BFGS:    2 15:45:02   -31079.442457        8.9026
BFGS:    3 15:45:58   -31085.021944       13.7504
BFGS:    4 15:46:48   -31088.730611       10.7427
BFGS:    5 15:47:30   -31091.476525        5.5919
BFGS:    6 15:48:05   -31093.273378        2.6185
BFGS:    7 15:48:31   -31094.529256        2.4301
BFGS:    8 15:48:54   -31095.282687        2.4310
BFGS:    9 15:49:17   -31096.104274        3.5226
BFGS:   10 15:49:43   -31096.899999        3.7859


In [106]:
# view([all_generated_strucutures_mol[15], mol_struc])

#### Writing out the structures to e.g. cif files

For any collection or list of atoms objects, we can save the structure out to a file using the ASE write function:


In [74]:
for i, s in enumerate(uniqueColl.structures):
    write(f'unique_single_Al-Na_{i:05d}.cif', s)
